# Машинное обучение (семинары)
## Урок 3. Классификация на практике

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.

Создайте модель линейной регрессии, которая бы оценивала лояльность пользователей к отклику.

Пояснения к признакам набора данных:

- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
- AGE — возраст клиента;
- SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- CHILD_TOTAL — количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- PERSONAL_INCOME — личный доход клиента (в рублях);
- LOAN_NUM_TOTAL — количество ссуд клиента;
- LOAN_NUM_CLOSED — количество погашенных ссуд клиента.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import resample
import pandas as pd

In [5]:
file_path = 'ClientsData.csv'
data = pd.read_csv(file_path)

data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AGE                15223 non-null  int64  
 1   SOCSTATUS_WORK_FL  15223 non-null  int64  
 2   SOCSTATUS_PENS_FL  15223 non-null  int64  
 3   GENDER             15223 non-null  int64  
 4   CHILD_TOTAL        15223 non-null  int64  
 5   DEPENDANTS         15223 non-null  int64  
 6   PERSONAL_INCOME    15223 non-null  float64
 7   LOAN_NUM_TOTAL     15223 non-null  int64  
 8   LOAN_NUM_CLOSED    15223 non-null  int64  
 9   LOAN_DLQ_NUM       15223 non-null  int64  
 10  TARGET             15223 non-null  int64  
dtypes: float64(1), int64(10)
memory usage: 1.3 MB


В дата фрейме отсутствуют пропуски

In [68]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
AGE,15223.0,40.406096,11.601068,21.0,30.0,39.0,50.0,67.0
SOCSTATUS_WORK_FL,15223.0,0.909610,0.286748,0.0,1.0,1.0,1.0,1.0
SOCSTATUS_PENS_FL,15223.0,0.134468,0.341165,0.0,0.0,0.0,0.0,1.0
GENDER,15223.0,0.654536,0.475535,0.0,0.0,1.0,1.0,1.0
CHILD_TOTAL,15223.0,1.099389,0.995411,0.0,0.0,1.0,2.0,10.0
DEPENDANTS,15223.0,0.645208,0.812252,0.0,0.0,0.0,1.0,7.0
PERSONAL_INCOME,15223.0,13853.836323,9015.467617,24.0,8000.0,12000.0,17000.0,250000.0
LOAN_NUM_TOTAL,15223.0,1.387769,0.793566,1.0,1.0,1.0,2.0,11.0
LOAN_NUM_CLOSED,15223.0,0.751889,0.988499,0.0,0.0,0.0,1.0,11.0
LOAN_DLQ_NUM,15223.0,0.195362,0.685022,0.0,0.0,0.0,0.0,13.0


In [69]:
# функция для вывода процентов по значениям признаков
def print_percentage(df,par_list):
    for par in par_list:
        print(df[par].value_counts())
        t_0, t_1 = df[par].value_counts()
        print('0: ',round((t_0 / (t_1 + t_0) * 100),2), '%')
        print('1: ',round(t_1 / (t_0 + t_1) * 100, 2), '%')
        print("-"*100)

In [71]:
list_parameters = ['GENDER', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL', 'TARGET']
print_percentage(data, list_parameters)

GENDER
1    9964
0    5259
Name: count, dtype: int64
0:  65.45 %
1:  34.55 %
----------------------------------------------------------------------------------------------------
SOCSTATUS_WORK_FL
1    13847
0     1376
Name: count, dtype: int64
0:  90.96 %
1:  9.04 %
----------------------------------------------------------------------------------------------------
SOCSTATUS_PENS_FL
0    13176
1     2047
Name: count, dtype: int64
0:  86.55 %
1:  13.45 %
----------------------------------------------------------------------------------------------------
TARGET
0    13411
1     1812
Name: count, dtype: int64
0:  88.1 %
1:  11.9 %
----------------------------------------------------------------------------------------------------


Анализ по столбцам:

- **AGE**: Возраст клиентов распределен между 21 и 50 годами, с некоторыми выбросами в сторону старших возрастов. Средний возраст около 40 лет.
- **SOCSTATUS_WORK_FL** и **SOCSTATUS_PENS_FL**: Большинство клиентов работают (91%) и не являются пенсионерами (87%).
- **GENDER**: Мужчин чуть больше, чем женщин.
- **CHILD_TOTAL** и **DEPENDANTS**: В среднем у клиентов чуть больше одного ребенка и одного иждивенца.
- **PERSONAL_INCOME**: Средний доход довольно высок, но есть значительный разброс, включая клиентов с очень низким доходом.
- **LOAN_NUM_TOTAL** и **LOAN_NUM_CLOSED**: В среднем у клиентов несколько кредитов, многие из которых уже погашены.
- **LOAN_DLQ_NUM**: Количество просроченных кредитов относительно невелико.
- **TARGET**: Только 12% клиентов откликнулись на предложение.

In [66]:
# Разделение данных на обучающую и тестовую выборки
X = data.drop(columns='TARGET')
y = data['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 1. Модель с взвешиванием классов
model_weighted = LogisticRegression(class_weight='balanced')
model_weighted.fit(X_train_scaled, y_train)

# Предсказания и оценка
y_pred_weighted = model_weighted.predict(X_test_scaled)
accuracy_weighted = accuracy_score(y_test, y_pred_weighted)
conf_matrix_weighted = confusion_matrix(y_test, y_pred_weighted)
class_report_weighted = classification_report(y_test, y_pred_weighted)

# 2. Ресемплирование данных: увеличение класса "Отклик"
X_train_resampled, y_train_resampled = resample(X_train[y_train == 1],
                                                y_train[y_train == 1],
                                                replace=True, 
                                                n_samples=X_train[y_train == 0].shape[0],
                                                random_state=42)

# Соединение ресемплированных и исходных данных класса 0
X_train_balanced = pd.concat([X_train[y_train == 0], X_train_resampled])
y_train_balanced = pd.concat([y_train[y_train == 0], y_train_resampled])

# Масштабирование ресемплированных данных
X_train_balanced_scaled = scaler.fit_transform(X_train_balanced)

# Обучение модели на сбалансированных данных
model_resampled = LogisticRegression()
model_resampled.fit(X_train_balanced_scaled, y_train_balanced)

# Предсказания и оценка
y_pred_resampled = model_resampled.predict(X_test_scaled)
accuracy_resampled = accuracy_score(y_test, y_pred_resampled)
conf_matrix_resampled = confusion_matrix(y_test, y_pred_resampled)
class_report_resampled = classification_report(y_test, y_pred_resampled)

# функция для вывода результатов
def print_results(model_name, accuracy, conf_matrix, class_report):
    print(f"\nМодель: {model_name}")
    print(f"Точность: {accuracy:.4f}")
    print("Матрица ошибок:")
    print(conf_matrix)
    print("\nОтчет по классификации:")
    print(class_report)

# Вывод результатов для обеих моделей
print_results("Взвешенная логистическая регрессия", accuracy_weighted, conf_matrix_weighted, class_report_weighted)
print_results("Логистическая регрессия с ресемплированием", accuracy_resampled, conf_matrix_resampled, class_report_resampled)



Модель: Взвешенная логистическая регрессия
Точность: 0.5934
Матрица ошибок:
[[1569 1107]
 [ 131  238]]

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.92      0.59      0.72      2676
           1       0.18      0.64      0.28       369

    accuracy                           0.59      3045
   macro avg       0.55      0.62      0.50      3045
weighted avg       0.83      0.59      0.66      3045


Модель: Логистическая регрессия с ресемплированием
Точность: 0.5215
Матрица ошибок:
[[1327 1349]
 [ 108  261]]

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.92      0.50      0.65      2676
           1       0.16      0.71      0.26       369

    accuracy                           0.52      3045
   macro avg       0.54      0.60      0.45      3045
weighted avg       0.83      0.52      0.60      3045



**Вывод:**
- Взвешенная модель имеет лучшую точность (59.34%) по сравнению с моделью с ресемплированием (52.15%), но страдает от низкого precision для откликов.
- Ресемплирование дало лучший результат по полноте откликов (71%), но значительно увеличило количество ложных срабатываний.